In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.CSV")

In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [6]:
dependent=dataset[['classification_yes']]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_= sc.fit_transform(x_train)
x_test_= sc.transform(x_test)

In [9]:
from sklearn.linear_model import LogisticRegression

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(x_train, y_train) 


Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [13]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.98490261799076


In [14]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[49  2]
 [ 0 82]]


In [15]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98        51
           1       0.98      1.00      0.99        82

    accuracy                           0.98       133
   macro avg       0.99      0.98      0.98       133
weighted avg       0.99      0.98      0.98       133



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9992826398852224

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.207461,0.251026,0.029182,0.009922,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.981569,0.981014,0.962573,0.961826,0.981031,0.973603,0.009316,2
1,0.513885,0.240461,0.024985,0.001413,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.869648,0.942166,0.906705,0.906085,0.981031,0.921127,0.037724,3
2,0.226660,0.242865,0.038777,0.014349,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.981569,0.981014,0.962573,0.962264,1.000000,0.977484,0.014072,1
3,0.410748,0.095178,0.027384,0.004961,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.486532,0.501410,0.477841,0.477841,0.477841,0.484293,0.009196,4


In [22]:
preinput=sc.transform([[51,70,0,0,219,36,1.3,139,3.7,12.5,49,6600,4.705,0,0,1,1,0,1,1,0,1,0,0,1,1,0]])

C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
preinput

array([[-0.06446668, -0.50842064, -0.68384276, -0.36409637,  0.90959604,
        -0.45754972, -0.32045843,  0.13552792, -0.39357677,  0.00972128,
         1.23095411, -0.71520424,  0.03647986, -0.49294082, -0.5454915 ,
         2.21610269,  7.22495675, -2.56904652,  0.51055216,  2.91547595,
        -0.23570226,  1.35269628, -0.73926425, -0.29277002,  0.51639778,
         1.91485422, -0.44519456]])

In [24]:
Future_Prediction=grid.predict(preinput) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
